# COGS 108 - EDA Checkpoint

# Names

- Jiayi Zhao
- Wenbo Hu
- Yunyi Huang
- Xiaotong Zeng

<a id='research_question'></a>
# Research Question

Is there a statistically significant relationship between the scale (burning area) of wildfire and climate variables in California that are associated with global warming such as relative humidity level, temperature and wind speed? Additionally, how can we utilize these climate variables to predict the wildfire event in California and the scale of wildfire?

# Setup

### Import Necessary Libraries

In [1]:
# Import pandas to read csv file and manage heterogenous data
import pandas as pd

# Import numpy to store numeric information and perform numerical analysis
import numpy as np

# Import seaborn and matplotlib to visualize data
import seaborn as sns
import matplotlib.pyplot as plt

# Import numpy to store numeric information and perform numerical analysis
import numpy as np

# Import seaborn and matplotlib to visualize data
import seaborn as sns
import matplotlib.pyplot as plt

#Import scipy to gather statistics
from scipy import stats

# Import patsy and statsmodels for regression analysis
import patsy
import statsmodels.api as sm

import warnings

import shutil

import os

### Import the three data sets that we need

In [ ]:
# Load the California wildfire incidents data set in data frame
# We get this data set from Kaggle (https://www.kaggle.com/ananthu017/california-wildfire-incidents-20132020)
wildfire = pd.read_csv("California_Fire_Incidents.csv")


# Load the US weather station ID data set in data frame
# We get the Integrated Surface Data (ISD) station list from ncdc.noaa.gov
station = pd.read_csv("https://www1.ncdc.noaa.gov/pub/data/noaa/isd-history.csv")


# Load the US weather daily data set from 2013 to 2019 in data frame
# We get this data from ncei.noaa.gov and download to the local.
# (https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/)
for dirname, _, filenames in os.walk('/Users/wenbohu/Desktop/Weather'):
    for filename in filenames:
        print((os.path.join(dirname, filename)))
        
# get all subdiretory of all tables
file_dict ={}
for path, dirs, files in os.walk('/Users/wenbohu/Desktop/Weather', topdown=False):
    file_dict[path]=files
    
paths = list(file_dict.keys())

events = []
for path in paths:
    events += [os.path.join(path,file) for file in file_dict[path]]
    

# Data Cleaning

Since we have three data set, we choose to clean them seperatly and then merge these dataset by locations.

### First, we clean the California wildfire incidents data set


Since we only need the dates, acres burned (scale), and county name for the following analysis, we update these information back to *'wildfire'*.

In [ ]:
# delete the irrelevant columns
wildfire = wildfire[['AcresBurned','Started','Counties', 'Latitude', 'Longitude']]

# change the started time into date
#wildfire['Started'] = pd.to_datetime(wildfire['Started'])
wildfire['Started'] = [x[0:10] for x in wildfire['Started']]

# change the 'Started' column name into 'Date'
wildfire = wildfire.rename({'Started':'Date'}, axis='columns')

#drop the null values 
wildfire['Latitude'] = wildfire['Latitude'].apply(lambda x: np.nan if x == 0 else x)
wildfire = wildfire.dropna().reset_index(drop=True) 


Now take a look on the *wildfire* dataframe

In [ ]:
wildfire.head()

### Second, clean the Integrated Surface Data (ISD) station list

In [ ]:
# Since the weather station ID is a combination of column 'USAF' and 'WBAN',
# we combine these two columns into a new column called 'ID'
station['ID']= station['USAF'].astype(str) + station['WBAN'].astype(str)

# we only analyze California weather
station = station[(station['STATE']=='CA') & (station['CTRY']=='US')].reset_index(drop=True)

# station only need to include the ID and the nameof the station
pd.set_option("max_rows", None)

Now take a look on the *station* dataframe

In [ ]:
station.head()

### Thrid, we merge the wildfire and station ID dataframes by matching the LATITUDE and LONGTITUDE of the wildfire incident locations and weather stations. 
We compare each error index (0.1, 0.2, 0.3, 0.5) in order to find which diameter we should choose for more unique stations is determined.

In [ ]:
IDlist = []
for i,j in wildfire.iterrows():
    before = len(IDlist)
    for a,b in station.iterrows():
        #about 50km * 40km (just for first time test then tried 0.3, 0.1, and 0.2)
        if (((b['LAT'] <= j['Latitude'] + 0.2) and (b['LAT'] >= j['Latitude'] - 0.2)) 
        and (( b['LON'] <= j['Longitude'] + 0.2) and ( b['LON'] >= j['Longitude'] - 0.2))):
            IDlist.append(b['ID'])
            break
    after = len(IDlist)
    if (before == after):
        IDlist.append("Not_Found")

In [ ]:
# 0.5-95 0.3-119 0.1-119 0.2-127(THIS IS THE BEST!!!!)
# when 0.1 it's also 119 but lots of not found values
unique = []
for x in IDlist:
    if x not in unique:
        unique.append(x)
print(len(unique))

Then, we create a dataframe called *matched_wildfire* that consists the scale, date, county name, latitude, and longitude of the wildfire incidents and the weather station ID in that incident area.

In [ ]:
IDlist = []
row_fire = []
row_ID =[]

# create a new dataframe to store these matched data
matched_wildfire = pd.DataFrame(columns=wildfire.columns)

# iterate the rows in wildfire and station to find the matched data
for i,j in wildfire.iterrows():
    for a,b in station.iterrows():
        if (((b['LAT'] <= j['Latitude'] + 0.2) and (b['LAT'] >= j['Latitude'] - 0.2)) 
        and (( b['LON'] <= j['Longitude'] + 0.2) and ( b['LON'] >= j['Longitude'] - 0.2))):
            IDlist.append(b['ID'])
            row_ID.append(b['ID'])
            row_fire.append(list(j))
            break
            
matched_wildfire = matched_wildfire.append(pd.DataFrame(row_fire,columns=wildfire.columns))
matched_wildfire = matched_wildfire.assign(ID=row_ID)

Now, take a look on the *matched_wildfire*

In [ ]:
matched_wildfire.head()

### Fourth, using the *matched_wildfire* data frame merge with the weather dataset from 2013 to 2019 so that every wildfire incident has the weather data of that day. 

Take a brief look on how the dataset of one station in the weather dataset in 2013 looks like

In [ ]:
example = pd.read_csv("/Users/wenbohu/Desktop/Weather/2013/40854099999.csv")
example.head()

Loop the *matched_wildfire* and find the weather data with the matched date and station ID

In [ ]:
# Get the weather information of wildfire start date 
row_weather = []
join_id = []
num = 1

# create a data frame to store the weather data
weather = pd.DataFrame(columns = example.columns)

for i, j in matched_wildfire.iterrows():
    for file in events:
        if num in join_id:
            break
        if (file[-15:-4] == j['Station_ID']):
            temp = pd.read_csv(file)
            for a, b in temp.iterrows():
                if (b["DATE"] == j['Date']):
                    row_weather.append(list(b))
                    join_id.append(num)
                    break 
    num += 1

Append these matched weather data to *weather*

In [ ]:
weather = pd.DataFrame(columns = example.columns)
weather = weather.append(pd.DataFrame(row_weather, columns = example.columns))
weather = weather.assign(Join_ID = join_id)

Now, take a look on the *weather*

In [ ]:
weather.head()

Merge *matched_wildfire* and *weather*, and store this merge dataset in local as a csv file

In [ ]:
dataframe = matched_wildfire.merge(weather, on = 'Join_ID')

In [ ]:
dataframe.to_csv('/Users/wenbohu/Desktop/df.csv')

### Lastly, import and clean this final data frame

The weather variable columns decription:
 - TMP: Mean temperature for the day in degrees Fahrenheit to tenths. Missing = 9999.9
 - DEWP: Mean dew point for the day in degreesm Fahrenheit to tenths. Missing = 9999.9
 - WDSD: Mean wind speed for the day in knots to tenths. Missing = 999.9
 - PRCP: Total precipitation (rain and/or melted snow) reported during the day in inches and hundredths; will usually not end with

In [2]:
df = pd.read_csv('df.csv')

df = df[['AcresBurned', 'Date', 'Counties','TEMP','DEWP','WDSP','ELEVATION', 'PRCP']] 

# By the column description, we replace the missing value (9999.9 or 999.9) with np.nan
df['TEMP'] = df['TEMP'].replace(9999.9, np.nan, regex=True) 
df['DEWP'] = df['DEWP'].replace(9999.9, np.nan, regex=True) 
df['WDSP'] = df['WDSP'].replace(999.9, np.nan, regex=True) 

# Drop the NAN value
df = df.dropna(subset=['TEMP','DEWP','WDSP'])

#### The Final Dataframe

In [3]:
df.head()

,AcresBurned,Date,Counties,TEMP,DEWP,WDSP,ELEVATION,PRCP
1,712.0,2013-07-19,Kern,80.5,33.8,13.2,1374.6,0.0
3,305.0,2013-10-04,Butte,66.2,27.4,10.1,57.9,0.0
4,298.0,2013-06-03,Butte,80.8,51.1,3.5,57.9,0.0
5,240.0,2013-06-08,Alameda,83.6,53.2,4.0,119.8,0.0
6,200.0,2013-07-04,Tehama,97.7,48.6,10.0,107.6,0.0


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [4]:
# understand (describe) what's going on in the data
df.describe()

,AcresBurned,TEMP,DEWP,WDSP,ELEVATION,PRCP
count,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000
mean,6147.862903,72.726882,46.010484,6.004301,279.641505,0.005376
std,42796.625726,9.469405,11.554669,2.880510,380.435179,0.074306
min,0.000000,36.700000,-1.000000,0.300000,1.800000,0.000000
25%,33.000000,66.275000,40.500000,4.300000,57.900000,0.000000
50%,78.000000,73.350000,48.400000,5.400000,119.800000,0.000000
75%,241.750000,79.225000,53.850000,6.900000,424.600000,0.000000
max,410203.000000,106.200000,74.600000,22.000000,2056.790000,1.360000
